<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/example_notebooks/11_slimyolov3/Train%20-%20Prune%20-%20Re-train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Table of contents

### 1. Installation

### 2. Training a Base Network
  - Training the base network
  - Running validation on base network to get accuracy
  - Running inference to get network speed on CPU

### 3. Pruning the Network + Finetuning
  - Pruning the trained base network
  - Finetuning the pruned network
  - Running validation on pruned network to get accuracy
  - Running inference to get network speed on CPU

# Installation

 - Run these commands
     
     - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git
     
     - cd Monk_Object_Detection/11_slimyolov3/installation
     
 - Select the right requirements file and run
 
     - cat requirements.txt | xargs -n 1 -L 1 pip install

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
# For colab use the command below
! cd Monk_Object_Detection/7_yolov3/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install

# For Local systems and cloud select the right CUDA version
# ! cd Monk_Object_Detection/7_yolov3/installation && cat requirements.txt | xargs -n 1 -L 1 pip install

# About the network

1. Paper on Yolov3: https://arxiv.org/abs/1804.02767

2. Paper on Yolov3-SPP: https://arxiv.org/abs/1903.08589

3. Paper on Slim-YoloV3: https://arxiv.org/abs/1907.11093

4. Darknet: https://pjreddie.com/darknet/

5. Blog-1 on yolo: https://machinethink.net/blog/object-detection-with-yolo/

6. Blog-2 on yolo: https://medium.com/@jonathan_hui/real-time-object-detection-with-yolo-yolov2-28b1b93e2088

7. Blog-3 on yolo: https://blog.ekbana.com/training-yolov2-in-a-custom-dataset-6fcf58f65fa2

8. Blog-4 on yolo: https://towardsdatascience.com/yolo-v3-object-detection-53fb7d3bfe6b

9. Blog-5 on yolo: https://blog.insightdatascience.com/how-to-train-your-own-yolov3-detector-from-scratch-224d10e55de2

# Monk Format

## Dataset Directory Structure

    ../sample_dataset/ship (root)
          |
          |-----------images/Train (img_dir)
          |              |
          |              |------------------img1.jpg
          |              |------------------img2.jpg
          |              |------------------.........(and so on)
          |
          |
          |-----------train_labels.csv (anno_file)
          
          
## Annotation file format

           | Id         | Labels                                 |
           | img1.jpg   | x1 y1 x2 y2 label1 x1 y1 x2 y2 label2  |
           
- Labels:  xmin ymin xmax ymax label
- xmin, ymin - top left corner of bounding box
- xmax, ymax - bottom right corner of bounding box

# Required Format - Yolo

## Dataset Directory Structure

    ../sample_dataset/ship (root)
          |
          |-----------images/Train (img_dir)
          |              |
          |              |------------------img1.jpg
          |              |------------------img2.jpg
          |              |------------------.........(and so on)
          |
          |-----------labels (label_dir)
          |              |
          |              |------------------img1.txt
          |              |------------------img2.txt
          |              |------------------.........(and so on)
          |
          |------------classes.txt 
          

## Classes file
 
     List of classes in every new line.
     The order corresponds to the IDs in annotation files
     
     Eg.
          class1               (------------------------------> if will be 0)
          class2               (------------------------------> if will be 1)
          class3               (------------------------------> if will be 2)
          class4               (------------------------------> if will be 3)
          

## Annotation file format

    CLASS_ID BOX_X_CENTER BOX_Y_CENTER WIDTH BOX_WIDTH BOX_HEIGHT
    
    (All the coordinates should be normalized)
    (X coordinates divided by width of image, Y coordinates divided by height of image)
    
    Ex. (One line per bounding box of object in image)
        class_id x1 y1 w h
        class_id x1 y1 w h
        ..... (and so on)
        

# Data Conversion

In [ ]:
import os
import sys

In [ ]:
root_dir = "Monk_Object_Detection/example_notebooks/sample_dataset/ship/";
img_dir = "images/Train";
anno_file = "train_labels.csv";

In [ ]:
labels_dir = "labels";
classes_file = "classes.txt";

In [ ]:
labels_dir_relative = root_dir + "/" + labels_dir
if(not os.path.isdir(labels_dir_relative)):
    os.mkdir(labels_dir_relative);

In [ ]:
import pandas as pd
df = pd.read_csv(root_dir + "/" + anno_file);
len(df)

In [ ]:
columns = df.columns
classes = [];
for i in range(len(df)):
    img_file = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(" ");
    for j in range(len(tmp)//5):
        label = tmp[j*5 + 4];
        if(label not in classes):
            classes.append(label);
classes = sorted(classes)
classes

In [ ]:
f = open(root_dir + "/" + classes_file, 'w');
for i in range(len(classes)):
    f.write(classes[i]);
    f.write("\n");
f.close();

In [ ]:
from PIL import Image
from tqdm.notebook import tqdm

In [ ]:
for i in tqdm(range(len(df))):
    img_file = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(" ");
    fname = labels_dir_relative + "/" + img_file.split(".")[0] + ".txt";
    img = Image.open(root_dir + "/" + img_dir + "/" + img_file);
    width, height = img.size
    
    f = open(fname, 'w');
    for j in range(len(tmp)//5):
        x1 = float(tmp[j*5 + 0]);
        y1 = float(tmp[j*5 + 1]);
        x2 = float(tmp[j*5 + 2]);
        y2 = float(tmp[j*5 + 3]);
        label = tmp[j*5 + 4];
        
        x_c = str(((x1 + x2)/2)/width);
        y_c = str(((y1 + y2)/2)/height);
        w = str((x2 - x1)/width);
        h = str((y2 - y1)/height);
        index = str(classes.index(label));
        
        f.write(index + " " + x_c + " " + y_c + " " + w + " " + h);
        f.write("\n");
    f.close();

# Training the base network with sparsity

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/11_slimyolov3/lib/")

In [ ]:
from train_detector import Detector

In [ ]:
gtf = Detector()

In [ ]:
img_dir = "Monk_Object_Detection/example_notebooks/sample_dataset/ship/Images/";
label_dir = "Monk_Object_Detection/example_notebooks/sample_dataset/ship/labels/";
class_list_file = "Monk_Object_Detection/example_notebooks/sample_dataset/ship/annotations/classes.txt";
gtf.set_train_dataset(img_dir, label_dir, class_list_file, batch_size=2, img_size=608)
gtf.set_val_dataset(img_dir, label_dir)

In [ ]:
os.path.isfile(class_list_file)

### Hyper-params for evolving params
  - Set evolve as True
  - Set num_generations (Total epochs = num_generations*epochs)

### Optimizers
  - "sgd"
  - "adam"

In [ ]:
gtf.set_hyperparams(optimizer="sgd", lr=0.0002, multi_scale=False, evolve=False, sparsity=0.0001);

In [ ]:
gtf.Train(num_epochs=2);

# Running Validation for base network

In [ ]:
import os
import sys
from IPython.display import Image
sys.path.append("Monk_Object_Detection/11_slimyolov3/lib");

In [ ]:
from infer_detector import Infer

In [ ]:
gtf = Infer();

In [ ]:
f = open("Monk_Object_Detection/example_notebooks/sample_dataset/ship/annotations/classes.txt");
class_list = f.readlines();
f.close();

In [ ]:
cfg_path = "yolov3-spp3.cfg"
weights = "weights/last.pt";
gtf.Model(cfg_path, class_list, weights, use_gpu=True, input_size=416);

In [ ]:
gtf.Validate(img_dir, label_dir, class_list)

# Running inference on CPU for base network

In [ ]:
import os
import sys
from IPython.display import Image
sys.path.append("Monk_Object_Detection/11_slimyolov3/lib");

In [ ]:
from infer_detector import Infer

In [ ]:
gtf = Infer();

In [ ]:
f = open("Monk_Object_Detection/example_notebooks/sample_dataset/ship/annotations/classes.txt");
class_list = f.readlines();
f.close();

In [ ]:
cfg_path = "yolov3-spp3.cfg"
weights = "weights/last.pt";
gtf.Model(cfg_path, class_list, weights, use_gpu=False, input_size=416);

In [ ]:
img_path = "Monk_Object_Detection/example_notebooks/sample_dataset/ship/test/img1.jpg";
gtf.Predict(img_path, conf_thres=0.01, iou_thres=0.5);
Image(filename='output/img1.jpg')

# Pruning the network and retraining

In [ ]:
import os
import sys
from IPython.display import Image
sys.path.append("Monk_Object_Detection/11_slimyolov3/lib");

In [ ]:
from train_detector import Detector

In [ ]:
gtf = Detector();

In [ ]:
gtf.prune_weights("yolov3-spp3.cfg", 
                  "weights/last.pt",
                  "pruned1.cfg", 
                  "pruned1.pt", 
                  img_size=608, 
                  overall_ratio=0.5, 
                  per_layer_ratio=0.1);

In [ ]:
img_dir = "Monk_Object_Detection/example_notebooks/sample_dataset/ship/Images/";
label_dir = "Monk_Object_Detection/example_notebooks/sample_dataset/ship/labels/";
class_list_file = "sample_dataset/ship/annotations/classes.txt";
gtf.set_train_dataset(img_dir, label_dir, class_list_file, batch_size=2, img_size=608)
gtf.set_val_dataset(img_dir, label_dir)

In [ ]:
gtf.set_hyperparams(optimizer="sgd", lr=0.0002, multi_scale=False, evolve=False); #keep sparsity=0

In [ ]:
gtf.set_finetune_params("weights/pruned1.cfg", 
                          "weights/pruned1.pt");

In [ ]:
gtf.Train(num_epochs=2, finetune=True)

# Running inference on pruned network

In [ ]:
import os
import sys
from IPython.display import Image
sys.path.append("Monk_Object_Detection/11_slimyolov3/lib");

In [ ]:
from infer_detector import Infer

In [ ]:
gtf = Infer();

In [ ]:
f = open("Monk_Object_Detection/example_notebooks/sample_dataset/ship/annotations/classes.txt");
class_list = f.readlines();
f.close();

In [ ]:
cfg_path = "weights/pruned1.cfg"
weights = "weights/last.pt";
gtf.Model(cfg_path, class_list, weights, use_gpu=False, input_size=416);

In [ ]:
img_path = "Monk_Object_Detection/example_notebooks/sample_dataset/ship/test/img1.jpg";
gtf.Predict(img_path, conf_thres=0.01, iou_thres=0.5);
Image(filename='output/img1.jpg')

# Running Validation for pruned network

In [ ]:
import os
import sys
from IPython.display import Image
sys.path.append("Monk_Object_Detection/11_slimyolov3/lib");

In [ ]:
from infer_detector import Infer

In [ ]:
gtf = Infer();

In [ ]:
f = open("Monk_Object_Detection/example_notebooks/sample_dataset/ship/annotations/classes.txt");
class_list = f.readlines();
f.close();

In [ ]:
cfg_path = "weights/pruned1.cfg"
weights = "weights/last.pt";
gtf.Model(cfg_path, class_list, weights, use_gpu=True, input_size=416);

In [ ]:
cfg_path = "yolov3-spp3.cfg"
weights = "weights/last.pt";
gtf.Model(cfg_path, class_list, weights, use_gpu=True, input_size=416);

In [ ]:
gtf.Validate(img_dir, label_dir, class_list)